In [ ]:
# Chainpoll User Guide

# How to run:

# 1. Open Google Colab and ensure you're signed in to your Google account.
# 2. Save the required files (general_data.json, qa_data.json, summarization_data.json, and Util.py) in your Google Drive under MyDrive.
# 3. Upload the chainpoll.ipynb file to Google Colab and open.
# 4. Change runtime type to T4 (GPU) for faster model processing. 
# 5. Click "Run" on each cell to execute the code sequentially.
#     - The code will:
#         - Install necessary packages
#         - Mount Google Drive to access files
#         - Load the model and data files from Google Drive
#         - Use the ChainPoll method to assess model responses for hallucinations
#         - Save results to a CSV file in Google Drive
# 6. Once complete, open your Google drive and locate the Chainpoll_qa_results.csv file. This file stores the prompts, responses, hallucination scores, and explanations in CSV format.
#    This can be opened in any spreadsheet software for analysis. 

# Note:
# - When prompted, enter your API key or access code for model access.
# - Adjust the for i, prompt in enumerate(prompts[1:25]): section to adjust how many prompts are ran for the qa_df dataset.
# - After execution, check your Google Drive (MyDrive) for "ChainPoll_qa_results.csv" to review the output.

In [ ]:
# Uninstall outdated library versions
!pip uninstall -y bitsandbytes accelerate

# Reinstall updated versions of required libraries
%pip install -U bitsandbytes accelerate
import bitsandbytes as bnb
import accelerate
import csv

# Mount Google Drive account to access files saved in the drive
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# Enabling import of custom module for colab
! cp /content/gdrive/MyDrive/Util.py . 

In [ ]:
# Set up file location here
import sys
sys.path.append('/content/gdrive/MyDrive')

# Import essential libraries used to run Chainpoll
from Util import load_data_JSON, load_model, write_out
import pandas as pd
import json
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

In [ ]:
# Load the data from JSON files for all tasks. Note, the qa_df is required only but consistency, all files are required. 
general_df, prompt_g, correct_g, gtp4_g, Info_g = load_data_JSON('/content/gdrive/MyDrive/general_data.json', 'general')
qa_df, prompt_q, correct_q, gtp4_q, Info_q = load_data_JSON('/content/gdrive/MyDrive/qa_data.json', 'qa')
sum_df, prompt_s, correct_s, gtp4_s, Info_s = load_data_JSON('/content/gdrive/MyDrive/summarization_data.json', 'sum')

In [ ]:
# Prompt for the user to input access code (API key, etc.)
access = input('Access code?')

In [ ]:
# Load the specified model and tokenizer using the access code for any necessary authentication
tokenizer, model = load_model("meta-llama/Llama-2-7b-chat-hf", access)

In [ ]:
# Set up a text generation pipeline with the model and tokenizer, using half-precision floats for efficiency
gen = pipeline("text-generation", model=model, torch_dtype=torch.float16, tokenizer=tokenizer)

In [ ]:
# Function to generate a single response for a given prompt
def get_response(prompt, max_len):
    sequences = gen(prompt, do_sample=True, top_k=5, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, max_length=max_len + len(prompt))
    return sequences[0]['generated_text']

In [ ]:
# ChainPoll method to assess responses for hallucinations
def chainpoll(model, tokenizer, prompts, num_responses=5):
    out_q = []

    # Loop through an amount of prompts (first 25 prompts)
    for i, prompt in enumerate(prompts[1:25]): # Adjust this based on how many prompts for the qa_df dataset you want to run
        hallucination_scores = []

        # Generate multiple responses per prompt to check for hallucinations
        for _ in range(num_responses):
            # Tokenize the prompt for input
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

            # Generate the response from the model
            with torch.no_grad():
                outputs = model.generate(
                    inputs['input_ids'],
                    max_new_tokens=50,  # Restrict the response length
                    num_beams=5,
                    early_stopping=True
                )

            # Decode the output response
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Ask the model if there are hallucinations using the CoT method
            hallucination_prompt = f"Does the following output contain hallucinations? Explain in detail:\n\nOutput: {response}\n"
            hallucination_inputs = tokenizer(hallucination_prompt, return_tensors="pt").to("cuda")

            # Generate the hallucination response
            with torch.no_grad():
                hall_outputs = model.generate(
                    hallucination_inputs['input_ids'],
                    max_new_tokens=100,  # Allow enough tokens for a full explanation
                    num_beams=5,
                    early_stopping=True
                )

            # Decode the hallucination response
            hallucination_response = tokenizer.decode(hall_outputs[0], skip_special_tokens=True)

            # Check for "yes" or "no" in the hallucination response and store the score
            if "yes" in hallucination_response.lower():
                hallucination_scores.append(1)
            else:
                hallucination_scores.append(0)

        # Compute final hallucination score as the proportion of "yes" responses
        hallucination_score = sum(hallucination_scores) / num_responses

         # Store the result with prompt, response, hallucination score, and explanation
        out_q.append({
            "prompt": prompt,
            "response": response,
            "hallucination_score": hallucination_score,
            "explanation": hallucination_response
        })

    return out_q


In [ ]:
# Execute the chainpoll function on the question and answering prompts
chainpoll_results = chainpoll(model, tokenizer, prompt_q)

In [ ]:
# Process and format the chainPoll results
out_q = []
for result in chainpoll_results:
    prompt = result["prompt"]
    response = result["response"]
    hallucination_score = result["hallucination_score"]
    explanation = result["explanation"]

    # Create a formatted string for each result
    formatted_output = f"""
    Prompt: {prompt}

    Response: {response}

    Hallucination Score: {hallucination_score}

    Explanation: {explanation}
    """
    
    # Append results to out_q for writing
    out_q.append(formatted_output)

# Optional - Print out the formatted results for checking (remove #'s)
# for formatted_result in out_q:
#    print(formatted_result)

In [ ]:
# Write out the formatted results to a CSV file in Google Drive
def write_out(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=',')  # Use a comma delimiter
        csv_writer.writerows([elt] for elt in data)

write_out("/content/gdrive/MyDrive/ChainPoll_qa_results.csv", out_q)